# Refgenie tutorial

The `Refgenie` class is a key component of the Refgenie package, which is used for managing and organizing reference genome files. It provides a set of methods and functionalities to interact with reference genome assets, and other related resources.

## Purpose of this file

This file serves as a tutorial for using the `Refgenie` Python API. It demonstrates how to set up a temporary directory for storing reference genome assets, configure the `Refgenie` instance, and perform various operations such as listing available assets, retrieving asset information, and managing data channels. 

In order to learn more about any of the contepts indicated in the code, please refer to a specific section of the documentation.

## Installation

Before the package gets released, clone the repository install, for example using `uv`:
  
```bash
git clone <repo_url>
cd refgenie1
uv pip install .
```

## Configuration

First, let's create a temporary directory that will be used to store the refgenie assets.


In [1]:
from pathlib import Path
from rich import print
import os

Let's set a temporary directory to store the refgenie assets.

In [2]:
from tempfile import TemporaryDirectory

REFGENIE_CODE_PATH = Path.cwd().parent / "refgenie" 

# set the environment variable
archive_tmp_dir = TemporaryDirectory(prefix="refgenie_archive_demo_").name
os.environ["REFGENIE_GENOME_ARCHIVE_FOLDER"] = archive_tmp_dir
tmp_dir = TemporaryDirectory(prefix="refgenie_demo_").name
os.environ["REFGENIE_GENOME_FOLDER"] = tmp_dir
# set the REFGENIE_DB_CONFIG_PATH to a sqlite config file in the refgenie package
# os.environ["REFGENIE_DB_CONFIG_PATH"] = (REFGENIE_CODE_PATH / "config" / "sqlite_config.yaml").as_posix()


Let's inspect the refgenie configuration object.

In [3]:
from refgenie.config import config

print(config)

RefgenieConfig(
    log_level=<LogLevel.INFO: 'INFO'>,
    genome_folder=PosixPath('/var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l'),
    genome_archive_folder=PosixPath('/var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_archive_demo_mthufo3
u'),
    database_config_path=PosixPath('/Users/stolarczyk/.refgenie/refgenie_db_config.yaml')
)

### Database backend

As you can see, refgenie configuration points to a database configuration file, as by default refgenie is backed by a SQLite database.

Let's inspect the refgenie database configuration file.

In [4]:
%cat {config.database_config_path}

type: sqlite
path: /Users/stolarczyk/.refgenie/refgenie


Make sure the directory where the SQLite database file is stored exists, and create it if it doesn't.

In [5]:
!refgenie1 purge --force
!rm -rf ~/refgenie_db
!mkdir -p ~/refgenie_db

INFO     Purged genome folder:                                   ]8;id=465630;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=670353;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#534\534]8;;\
         '/var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refge                
         nie_demo_ig9f7if_'                                                     
INFO     Purged refgenie backend:                                ]8;id=559086;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=833022;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#536\536]8;;\
         'sqlite:////Users/stolarczyk/.refgenie/refgenie'                       


In practice, you don't even need to create the configuration file manually, as refgenie ships with a default configuration file that is used if no configuration file is provided. Just as we've seen above.

For production deployments you may want to use a different database backend, such as MySQL or PostgreSQL. In this case, you can provide the database configuration file path by setting `REFGENIE_DB_CONFIG_PATH` environment variable, or even set/override the database engine using `database_engine` in the `Refgenie` constructor. The object must be a `sqlalchemy.engine.Engine` object.



### Refgenieserver client

Similarly, refgenie ships with a Refgenieserver client, which is used by default to retrieve remote genome assets and does not need to be replaced in majority of use cases. However, you can provide a custom URL-client mapping to `Refgenie` constructor, by setting the `server_client_mapping` argument. Please note that, the clients need to follow a specific interface, defined in `refgenie.server.ServerClient` protocol. More details below.

In [6]:
from refgenie.server.models import ServerClient
from rich import inspect

inspect(
    ServerClient,
    methods=True,
    docs=True,
    help=True,
    title="ServerClient Protocol structure",
)

╭──────────────────────────────────────── ServerClient Protocol structure ────────────────────────────────────────╮
│ class ServerClient(*args, **kwargs):                                                                            │
│                                                                                                                 │
│ Protocol for the server client.                                                                                 │
│                                                                                                                 │
│             server_url = <property object at 0x111798e00>                                                       │
│ download_with_progress = def download_with_progress(self, operation_id: str, output_path: pathlib.Path, params: │
│                          Optional[Dict] = None, url_format_params: Optional[Dict[str, str]] = None, name:       │
│                          Optional[str] = None) -> pathlib.Path: Download asset at given URL to given filepath,  │
│                          show progress along the way.                                                           │
│                    get = def get(self, operation_id: str, params: Optional[Dict] = None, url_format_params:     │
│                          Optional[Dict[str, str]] = None) -> Dict: Send a GET request to the specified          │
│                          operation ID.                                                                          │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

First, let's import the `Refgenie` class from the `refgenie` package.


In [7]:
from refgenie import Refgenie

refgenie = Refgenie(suppress_migrations=True)

Let's ensure we start with a clean slate by removing any existing refgenie metadata and initializing a new refgenie instance.


In [8]:
refgenie.init()  # initialize new refgenie instance

INFO     Initialized refgenie backend: 'sqlite:////Users/stolarczyk/.refgenie/refgenie'             ]8;id=429936;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=483822;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#392\392]8;;\

INFO     Registered 'fasta' asset class                                                               ]8;id=422903;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py\manager.py]8;;\:]8;id=383866;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py#71\71]8;;\

INFO     Registered 'fasta' recipe                                                                   ]8;id=79091;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py\manager.py]8;;\:]8;id=135854;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py#113\113]8;;\

Let's subscribe to the default refgenie server. This method will reach out to the server at the provided URL and query the OpenAPI specification to determine whether ther server is refgenie-compatible. If it is, the server will be added to the list of subscribed servers.

Note: there's currently no public compatible refgenieserver instance deployed, so the following code snippets use a local refgenieserver instance serving the latest API.


In [9]:
refgenie.configuration.subscribe("http://localhost:8000")

INFO     Subscribed to servers: http://localhost:8000                                                 ]8;id=908093;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/configuration/manager.py\manager.py]8;;\:]8;id=103925;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/configuration/manager.py#42\42]8;;\

And that's it! We have now configured a refgenie instance and subscribed to a refgenie-compatible server. We can now start using the refgenie instance to manage reference genome assets.

### Pull an asset

Let's initialize a new genome by pulling an asset of fasta class. This will create a new directory in the `data` subdirectory of the `genome_folder` and mirror it in the `alias` directory with symbolic links, rather than copies of the files.


In [10]:
refgenie.pull(alias_name="rCRSd-1", asset_group_name="fasta")

INFO     HTTP Request: GET http://localhost:8000/openapi.json "HTTP/1.1 200 OK"                     ]8;id=843848;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=88946;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/httpx/_client.py#1027\1027]8;;\

INFO     Connected to server: title='Refgenieserver REST API' version='1.0.0' description='a web       ]8;id=864197;file:///Users/stolarczyk/code/refgenie1/refgenie/server/client.py\client.py]8;;\:]8;id=38338;file:///Users/stolarczyk/code/refgenie1/refgenie/server/client.py#32\32]8;;\
         interface and RESTful API for reference genome assets'                                                    

WARNING  No local digest for genome alias: rCRSd-1. Setting genome identity with server:           ]8;id=904203;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=213246;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#3124\3124]8;;\
         http://localhost:8000                                                                                     

INFO     Setting 'rCRSd-1' identity with server: http://localhost:8000                             ]8;id=416443;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=279008;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1070\1070]8;;\

INFO     HTTP Request: GET http://localhost:8000/v4/aliases/rCRSd-1 "HTTP/1.1 200 OK"               ]8;id=979389;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=839055;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/httpx/_client.py#1027\1027]8;;\

...

INFO     Initialized genome: ZtAkf32sCUjeSl0KxVA5DVevklHDazQM (Yeast genome 1)                     ]8;id=139939;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=280730;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1272\1272]8;;\

INFO     Added: 'rCRSd-1/fasta:samtools-1.21'                                                       ]8;id=301067;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=226831;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#773\773]8;;\

INFO     Created alias directories:                                                                ]8;id=82741;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=619609;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#3515\3515]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/alias/rCRSd-1/fas                 
         ta/samtools-1.21                                                                                          

INFO     Set default asset: 'ZtAkf32sCUjeSl0KxVA5DVevklHDazQM/fasta:samtools-1.21'                 ]8;id=285797;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=441116;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1640\1640]8;;\

Asset(name='samtools-1.21', description=None, size=34082, updated_at=datetime.datetime(2025, 7, 6, 18, 28, 50, 885001), path='data/ZtAkf32sCUjeSl0KxVA5DVevklHDazQM/fasta/samtools-1.21', digest='8ccce3f01185ef75c8dabeb9e03f8822', recipe_id=None, asset_group_id=1, created_at=datetime.datetime(2025, 7, 6, 18, 28, 50, 885007))

As you can see above, the genome has been initialized and `fasta` asset was pulled. Let's inspect the initialized genome.


In [11]:
print(refgenie.genomes_table())

                       Genomes                       
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Digest                           ┃ Description    ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ ZtAkf32sCUjeSl0KxVA5DVevklHDazQM │ Yeast genome 1 │
└──────────────────────────────────┴────────────────┘

Now, that a `fasta` asset has been built for the `dm6` genome, let's add some custom asset classes and recipes to build an asset based on that.

### Add `bowtie2_index` asset class and recipe

By supplying a URL (`str` object) rather than a local path (`pathlib.Path` object), refgenie will grab the remote file and register it as if it was a local file.


In [12]:
refgenie.asset_class.add(
    "https://github.com/refgenie/recipes/raw/refgenie1/asset_classes/bowtie2_index_asset_class.yaml"
)
refgenie.recipe.add(
    "https://github.com/refgenie/recipes/raw/refgenie1/recipes/bowtie2_index_asset_recipe.yaml"
)

INFO     Registered 'bowtie2_index' asset class                                                       ]8;id=559618;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py\manager.py]8;;\:]8;id=47268;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py#71\71]8;;\

INFO     Registered 'bowtie2_index' recipe                                                           ]8;id=872041;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py\manager.py]8;;\:]8;id=899989;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py#113\113]8;;\

Recipe(id=2, name='bowtie2_index', version='0.0.1', description='Genome index for bowtie2, produced with bowtie2-build', output_asset_class_id=2, command_templates=['bowtie2-build --threads {{values.params["threads"]}} {{values.genome_folder}}/{{values.assets["fasta"].seek_keys_dict["fasta"]}} {{values.output_folder}}/{{values.genome_digest}}'], input_params={'threads': {'description': 'Number of threads to use', 'default': 1}}, input_files=None, input_assets={'fasta': {'asset_class': 'fasta', 'description': 'fasta asset for genome', 'default': 'fasta'}}, docker_image='docker.io/databio/refgenie', custom_properties={'version': "bowtie2-build --version | awk 'NR==1{print $3}'"}, default_asset='{{values.custom_properties.version}}', updated_at=datetime.datetime(2025, 7, 6, 18, 28, 51, 196749), created_at=datetime.datetime(2025, 7, 6, 18, 28, 51, 196751))

Let's verify that it worked by listing the available asset classes and recipes:


In [13]:
from rich import print

print(refgenie.recipe.table())
print(refgenie.asset_class.table())

                                                      Recipes                                                      
┏━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃               ┃         ┃ Output asset    ┃ Input asset    ┃                 ┃                ┃                 ┃
┃ Name          ┃ Version ┃ class           ┃ classes        ┃ Input files     ┃ Input params   ┃ Docker image    ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ fasta         │ 0.0.1   │ fasta           │ None           │ • fasta (DNA    │ None           │ docker.io/data… │
│               │         │                 │                │ sequences in    │                │                 │
│               │         │                 │                │ the FASTA       │                │                 │
│               │         │                 │                │ format)         │                │                 │
│               │         │                 │                │ default=None    │                │                 │
├───────────────┼─────────┼─────────────────┼────────────────┼─────────────────┼────────────────┼─────────────────┤
│ bowtie2_index │ 0.0.1   │ bowtie2_index   │ • fasta (fasta │ None            │ • threads      │ docker.io/data… │
│               │         │                 │ asset for      │                 │ (Number of     │                 │
│               │         │                 │ genome)        │                 │ threads to     │                 │
│               │         │                 │ default=fasta  │                 │ use) default=1 │                 │
└───────────────┴─────────┴─────────────────┴────────────────┴─────────────────┴────────────────┴─────────────────┘

                                                   Asset Classes                                                   
┏━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name          ┃ Version ┃ Seek keys               ┃ Description                                                 ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ fasta         │ 0.0.1   │ fasta, fai, chrom_sizes │ Sequences in the FASTA format, indexed FASTA (produced with │
│               │         │                         │ samtools index) and chromosome sizes file                   │
├───────────────┼─────────┼─────────────────────────┼─────────────────────────────────────────────────────────────┤
│ bowtie2_index │ 0.0.1   │ bowtie2_index           │ Genome index for bowtie2, produced with bowtie2-build       │
└───────────────┴─────────┴─────────────────────────┴─────────────────────────────────────────────────────────────┘

## Build a `fasta` asset


In [14]:
from refgenie import BuildParams

refgenie.build_asset(
    recipe_name="fasta",
    genome_name="t7",
    asset_group_name="fasta",
    params=BuildParams(
        files={"fasta": REFGENIE_CODE_PATH.parent / "tests/data/t7.fa"}
    ),
    genome_description="Genome of T7 phage",
)

INFO     Initializing genome from FASTA file: /Users/stolarczyk/code/refgenie1/tests/data/t7.fa    ]8;id=86306;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=155030;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1238\1238]8;;\

INFO     SeqCol: digest='kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB'                                            ]8;id=166147;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/refget/agents.py\agents.py]8;;\:]8;id=313169;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/refget/agents.py#242\242]8;;\
         sorted_name_length_pairs_digest='0R51wR5l44VrpWoLqmcfyYcrLLba--uY'                                        

INFO     Initialized genome: kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB (Genome of T7 phage)                 ]8;id=692756;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=783966;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1272\1272]8;;\

INFO     Building 't7/fasta' using recipe 'fasta (v0.0.1)'                                         ]8;id=540725;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=264622;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#2499\2499]8;;\

Using default schema: /Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/pipestat_output_schema.yaml


...

INFO     Created alias directories:                                                                ]8;id=376592;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=681535;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#3515\3515]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/alias/t7/fasta/sa                 
         mtools-1.21                                                                                               

INFO     Set default asset: 'kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/fasta:samtools-1.21'                 ]8;id=217703;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=775800;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1640\1640]8;;\

INFO     Added asset: kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/fasta:samtools-1.21                         ]8;id=964619;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=767719;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#2618\2618]8;;\

INFO     Updated parents of 'kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/fasta:samtools-1.21'                 ]8;id=571774;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=213956;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#2856\2856]8;;\

Asset(name='samtools-1.21', description='DNA sequences in the FASTA format, indexed FASTA (produced with samtools index), chromosome sizes file and FASTA dict (produced with samtools dict)', size=42981, updated_at=datetime.datetime(2025, 7, 6, 18, 28, 51, 515997), path='data/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/fasta/samtools-1.21', digest='51a58ef25e4f1d7e76f226fd5655754f', recipe_id=1, asset_group_id=2, created_at=datetime.datetime(2025, 7, 6, 18, 28, 51, 516002))

### Build a `bowtie2_index` asset

The `bowtie2_index` asset class and recipe have been added successfully. Let's build the `bowtie2_index` asset for the `dm6` genome.


In [15]:
from refgenie.models import BuildParams

refgenie.build_asset(
    recipe_name="bowtie2_index",
    genome_name="t7",
    asset_group_name="bowtie2_index",
    params=BuildParams(params={"threads": 8}),
    archive=True,  # archive the asset right after building
)

INFO     Building 't7/bowtie2_index' using recipe 'bowtie2_index (v0.0.1)'                         ]8;id=333031;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=211929;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#2499\2499]8;;\

Using default schema: /Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/pipestat_output_schema.yaml


INFO     Could not locate config file.                                                         ]8;id=705392;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/yacman/yacman_future.py\yacman_future.py]8;;\:]8;id=371662;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/yacman/yacman_future.py#563\563]8;;\

No pipestat output schema was supplied to PipestatManager.
Initializing results file '/var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/data/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/bowtie2_index/2.5.3/_refgenie_build/stats.yaml'


INFO     File does not exist, but create_file is true. Creating...                              ]8;id=125264;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/ubiquerg/file_locking.py\file_locking.py]8;;\:]8;id=250071;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/ubiquerg/file_locking.py#251\251]8;;\

...

building file list ... done
kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.1.bt2
kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.2.bt2
kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.3.bt2
kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.4.bt2
kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.rev.1.bt2
kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.rev.2.bt2

sent 8447138 bytes  received 152 bytes  16894580.00 bytes/sec
total size is 8445686  speedup is 1.00


INFO     Running: tar --exclude '_refgenie_build' -C                                                ]8;id=930526;file:///Users/stolarczyk/code/refgenie1/refgenie/utils/archiving.py\archiving.py]8;;\:]8;id=628552;file:///Users/stolarczyk/code/refgenie1/refgenie/utils/archiving.py#19\19]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/data/kN9XHLKLS_u7e                
         i2GH87H-qpQrkz8moPB/bowtie2_index -cvf - 2.5.3 | pigz >                                                   
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_archive_demo_mthufo3u/kN9XHLKLS_                
         u7ei2GH87H-qpQrkz8moPB/bowtie2_index/2.5.3/bowtie2_index__2.5.3.tgz                                       

a 2.5.3
a 2.5.3/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.4.bt2
a 2.5.3/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.rev.2.bt2
a 2.5.3/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.2.bt2
a 2.5.3/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.3.bt2
a 2.5.3/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.1.bt2
a 2.5.3/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.rev.1.bt2


INFO     Created asset archive at:                                                                    ]8;id=85231;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/archive/manager.py\manager.py]8;;\:]8;id=110289;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/archive/manager.py#85\85]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_archive_demo_mthufo3u/kN9XHLKLS_u7              
         ei2GH87H-qpQrkz8moPB/bowtie2_index/2.5.3/bowtie2_index__2.5.3.tgz                                         

Asset(name='2.5.3', description='Genome index for bowtie2, produced with bowtie2-build', size=8446757, updated_at=datetime.datetime(2025, 7, 6, 18, 28, 51, 871146), path='data/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/bowtie2_index/2.5.3', digest='40a7b72a358850f722b6734a836b0fa8', recipe_id=2, asset_group_id=3, created_at=datetime.datetime(2025, 7, 6, 18, 28, 51, 871153))

Let's list the assets for the genome `t7` to verify that the `bowtie2_index` asset has been built successfully.


In [16]:
refgenie.assets_table(genome_names=["t7"])[0]

                   Refgenie assets. Source: local                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Genome digest                    ┃ Asset group   ┃ Asset         ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB │ fasta         │ samtools-1.21 │
│ kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB │ bowtie2_index │ 2.5.3         │
└──────────────────────────────────┴───────────────┴───────────────┘

One of the assets was also archived (a neccessary step to serve the assets via the refgenie server). Let's list the archived assets.

In [17]:
print(refgenie.archive.table())

                                                  Asset Archives                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ Digest                          ┃ Asset name                      ┃ Path                            ┃ Size      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━┩
│ ddf6f41d9d21d404fe5c7f07adb2f2… │ kN9XHLKLS_u7ei2GH87H-qpQrkz8mo… │ /var/folders/18/3fc3jyt50sv9kq… │ 230.38 KB │
└─────────────────────────────────┴─────────────────────────────────┴─────────────────────────────────┴───────────┘

Asset `bowtie2_index` has been built successfully for the `dm6` genome, and automatically tagged with `2.5.3`, indicating the version of Bowtie2 software used (this behavior is encoded in the recipe).


## Interact with aliases

Let's list the aliases:

In [18]:
refgenie.aliases_table()

                           Genome aliases                            
┏━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Alias(es) ┃ Genome digest                    ┃ Genome description ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ rCRSd-1   │ ZtAkf32sCUjeSl0KxVA5DVevklHDazQM │ Yeast genome 1     │
│ t7        │ kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB │ Genome of T7 phage │
└───────────┴──────────────────────────────────┴────────────────────┘

Let's assign another alias to the same genome digest, this way we can refer to the same genome in multiple ways.

In [19]:
t7_alias = refgenie.set_genome_alias(
    alias_name="Bacteriophage-T7",
    genome_digest="kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB",
    genome_description="My favorite genome",
)
fav_alias = refgenie.set_genome_alias(
    alias_name="myFavGenome",
    genome_digest="kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB",
    genome_description="My favorite genome",
)

INFO     Added alias: Bacteriophage-T7                                                             ]8;id=211677;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=17658;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1153\1153]8;;\

INFO     Created alias directories:                                                                ]8;id=652504;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=41561;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#3515\3515]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/alias/Bacteriopha                 
         ge-T7                                                                                                     

INFO     Added alias: myFavGenome                                                                  ]8;id=684505;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=419575;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1153\1153]8;;\

INFO     Created alias directories:                                                                ]8;id=7727;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=803708;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#3515\3515]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/alias/myFavGenome                 

The new alias should be listed in the aliases:

In [20]:
refgenie.aliases_table()

                                       Genome aliases                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ Alias(es)                         ┃ Genome digest                    ┃ Genome description ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ rCRSd-1                           │ ZtAkf32sCUjeSl0KxVA5DVevklHDazQM │ Yeast genome 1     │
│ t7, Bacteriophage-T7, myFavGenome │ kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB │ Genome of T7 phage │
└───────────────────────────────────┴──────────────────────────────────┴────────────────────┘

The command not only creates a new alias, but also creates a symbolic links to the files in the `data` directory for that genome.

Conversely, alias removal will remove the symbolic links, but not the files in the `data` directory.

In [21]:
refgenie.remove_alias("myFavGenome")

INFO     Deleting alias files:                                                                         ]8;id=487449;file:///Users/stolarczyk/code/refgenie1/refgenie/db/events.py\events.py]8;;\:]8;id=56337;file:///Users/stolarczyk/code/refgenie1/refgenie/db/events.py#49\49]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/alias/myFavGenome                 

INFO     Removed alias: myFavGenome                                                                ]8;id=843484;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=742516;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#1401\1401]8;;\

## Retrieve paths to assets

Most importantly, we can retrieve paths to refgenie-managed files.

All below commands will return the same path to the fasta file managed by Refgenie:

In [22]:
print(refgenie.seek("t7", "fasta"))
print(refgenie.seek("Bacteriophage-T7", "fasta", "samtools-1.21"))
print(refgenie.seek("t7", "fasta", "samtools-1.21", "fasta"))

/var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/data/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/fasta
/samtools-1.21/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.fa

/var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/data/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/fasta
/samtools-1.21/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.fa

/var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/data/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/fasta
/samtools-1.21/kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB.fa

## Remove an asset

Let's remove the `bowtie2_index` asset for the `dm6` genome.

In [23]:
refgenie.remove_asset_group(
    genome_name="t7", asset_group_name="bowtie2_index", force=True
)

INFO     Deleting archive files:                                                                       ]8;id=188691;file:///Users/stolarczyk/code/refgenie1/refgenie/db/events.py\events.py]8;;\:]8;id=56642;file:///Users/stolarczyk/code/refgenie1/refgenie/db/events.py#59\59]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_archive_demo_mthufo3u/kN9XHLKLS_u7e             
         i2GH87H-qpQrkz8moPB/bowtie2_index/2.5.3                                                                   

INFO     Deleting asset files:                                                                         ]8;id=400807;file:///Users/stolarczyk/code/refgenie1/refgenie/db/events.py\events.py]8;;\:]8;id=356769;file:///Users/stolarczyk/code/refgenie1/refgenie/db/events.py#32\32]8;;\
         /var/folders/18/3fc3jyt50sv9kqx6hdqg5b600000gn/T/refgenie_demo_ugm8n45l/data/kN9XHLKLS_u7ei2G             
         H87H-qpQrkz8moPB/bowtie2_index/2.5.3                                                                      

INFO     Removed asset group and all assets 'kN9XHLKLS_u7ei2GH87H-qpQrkz8moPB/bowtie2_index'        ]8;id=967535;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py\refgenie.py]8;;\:]8;id=725790;file:///Users/stolarczyk/code/refgenie1/refgenie/refgenie.py#919\919]8;;\

## Data channels

Refgenie supports data channels, which are used to allow third-party tool developers to expose their recipes and asset classes to Refgenie ecosystem.
In the simplest case, data channels is just a github repository with an index file that lists available asset classes and recipes, like so:

```yaml
asset_class:
  dir: asset_classes # optional, needed only if the asset classes are stored in a subdirectory
  files: # list of asset class files, relative to the index file (or directory)
    - fasta.yaml 
    - bowtie2_index.yaml
recipe:
  dir: recipes # optional, needed only if the recipes are stored in a subdirectory
  files: # list of recipe files, relative to the index file (or directory)
    - fasta.yaml
    - bowtie2_index.yaml
```

One such example is the [refgenie/recipes](https://github.com/refgenie/recipes/blob/refgenie1/index.yaml) repository, which can be added as a data channel to refgenie in the following way:

In [24]:
data_channel = refgenie.data_channel.add(
    name="refgenie-recipes",
    type="http",
    index_address="https://refgenie.github.io/recipes/index.yaml",
    description="Refgenie recipes channel",
)

print(refgenie.data_channel.table())

                                                   Data Channels                                                   
┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Name             ┃ Type ┃ Index Address                            ┃ Description              ┃ Credentials set ┃
┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ refgenie-recipes │ http │ https://refgenie.github.io/recipes/inde… │ Refgenie recipes channel │ False           │
└──────────────────┴──────┴──────────────────────────────────────────┴──────────────────────────┴─────────────────┘

Subsequently, the asset classes and recipes from the data channel can be listed and added to the refgenie instance.

In [25]:
for asset_class in refgenie.data_channel.iter_asset_classes("refgenie-recipes"):
    try:
        refgenie.asset_class.add(asset_class)
    except Exception as e:
        print(e)

for recipe in refgenie.data_channel.iter_recipes("refgenie-recipes"):
    try:
        refgenie.recipe.add(recipe)
    except Exception as e:
        print(e)
        


INFO     HTTP Request: GET https://refgenie.github.io/recipes/index.yaml "HTTP/1.1 200 OK"          ]8;id=557378;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=826819;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/httpx/_client.py#1027\1027]8;;\

INFO     Registered 'abundant_sequences' asset class                                                  ]8;id=330617;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py\manager.py]8;;\:]8;id=892320;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py#71\71]8;;\

INFO     Registered 'bed' asset class                                                                 ]8;id=69834;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py\manager.py]8;;\:]8;id=702813;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py#71\71]8;;\

INFO     Registered 'bismark_bt1_index' asset class                                                   ]8;id=642076;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py\manager.py]8;;\:]8;id=628252;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py#71\71]8;;\

INFO     Registered 'bismark_bt2_index' asset class                                                   ]8;id=144701;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py\manager.py]8;;\:]8;id=349081;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/asset_class/manager.py#71\71]8;;\

...

INFO     Registered 'small_rna' recipe                                                               ]8;id=491164;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py\manager.py]8;;\:]8;id=8782;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py#113\113]8;;\

INFO     Registered 'star_index' recipe                                                              ]8;id=601788;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py\manager.py]8;;\:]8;id=759285;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py#113\113]8;;\

INFO     Registered 'suffixerator_index' recipe                                                      ]8;id=864134;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py\manager.py]8;;\:]8;id=136058;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py#113\113]8;;\

INFO     Registered 'tallymer_index' recipe                                                          ]8;id=430821;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py\manager.py]8;;\:]8;id=8618;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py#113\113]8;;\

INFO     Registered 'tgMap' recipe                                                                   ]8;id=998292;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py\manager.py]8;;\:]8;id=89803;file:///Users/stolarczyk/code/refgenie1/refgenie/resource_manager/recipe/manager.py#113\113]8;;\

Alternatively, the same can be achieved by running the following CLI command:

```bash
refgenie1 data_channel sync refgenie-recipes --exists-ok
```

## SeqCol interface

Refgenie also provides a `SeqCol` interface, which is standard for working with sequence collections. More details on this interface can be found on the [SeqCol project website](https://seqcol.readthedocs.io/en/latest/).
Under the hood, refgenie uses the `SeqCol` digests to uniquely identify genomes.

In [26]:
d1 = refgenie.refget_db_agent.seqcol.add_from_fasta_file(
    "/Users/stolarczyk/code/refgenie1/tests/data/rCRSd.fa"
).digest
d2 = refgenie.refget_db_agent.seqcol.add_from_fasta_file(
    "/Users/stolarczyk/code/refgenie1/tests/data/rCRSd-extra.fa"
).digest
refgenie.refget_db_agent.compare_digests(d1, d2)

INFO     SeqCol: digest='jthDpfNIgzM5AGJlOkRtfnky4rXMBIUP'                                            ]8;id=546164;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/refget/agents.py\agents.py]8;;\:]8;id=793969;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/refget/agents.py#242\242]8;;\
         sorted_name_length_pairs_digest='AYk42eFmfBv_Q3GRVXWRKv2BWJ-2rWaN'                                        

INFO     SeqCol: digest='smiTbD3jP5EwF4DNWVm0c6DGKRlFHfas'                                            ]8;id=433183;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/refget/agents.py\agents.py]8;;\:]8;id=67038;file:///Users/stolarczyk/code/refgenie1/.venv/lib/python3.12/site-packages/refget/agents.py#242\242]8;;\
         sorted_name_length_pairs_digest='C4yiaCS1eR4cxiQCg4tInGAIiJ8rG6UG'                                        

{'attributes': {'a_only': [],
  'b_only': [],
  'a_and_b': ['lengths',
   'name_length_pairs',
   'names',
   'sequences',
   'sorted_sequences']},
 'array_elements': {'a': {'lengths': 1,
   'name_length_pairs': 1,
   'names': 1,
   'sequences': 1,
   'sorted_sequences': 1},
  'b': {'lengths': 2,
   'name_length_pairs': 2,
   'names': 2,
   'sequences': 2,
   'sorted_sequences': 2},
  'a_and_b': {'lengths': 1,
   'name_length_pairs': 1,
   'names': 1,
   'sequences': 1,
   'sorted_sequences': 1},
  'a_and_b_same_order': {'lengths': True,
   'name_length_pairs': True,
   'names': True,
   'sequences': True,
   'sorted_sequences': True}}}